In [147]:
import pandas as pd
import np
from sklearn.utils import shuffle
from math import ceil

In [148]:
input_filename = "/home/mchughj/Downloads/Team Placement - Form Responses 1.csv"
all_players = pd.read_csv(input_filename)

print( f"Read in the player file; players: {len(all_players)}")

mindy = all_players[all_players["Name"] == "Mindy Au"]

Read in the player file; players: 100


In [149]:
class Team:
    def __init__(self, n, g):
        self.n = n
        assert(isinstance(g, pd.core.series.Series))
        self.g = g
        self.tm = []
        self.o = []
        self.d = []
        self.m = []
        
    def _getPlayerName(self, p):
        return p["Name"].strip()
    
    def addPlayer(self, p):
        if isinstance(p, pd.core.frame.DataFrame): 
            assert(len(p) == 1)
            p = p.iloc[0]
        assert(isinstance(p, pd.core.series.Series))
        self.tm.append(p)
        
    def _chooseRandom(self, number, allPlayers, matching, exclude):
        allPlayers = shuffle(allPlayers)
        result = []
        # print(f"_ChooseRandom - looking for {number} of {matching} but exclusion is size {len(exclude)} - first 3 people {allPlayers[0]['Name']}, {allPlayers[1]['Name']}, {allPlayers[2]['Name']},")
        for person in allPlayers: 
            # print(f"Considering person: {person['Name']}")
            # print(f"Against exclude: {exclude}")
            
            excluded = False
            for e in exclude:
                if e["Name"] == person["Name"]:
                    excluded = True
                    
            if excluded:
                continue
            
            availablePositions = person["Position"].split(",")
            if availablePositions[0] == "No preference":
                availablePositions = ["Offense", "Defense", "Midfield"]
            
            if matching in availablePositions:
                # print(f"Match found in {person['Name']}: looking: {matching}, found: {person['Position']}")
                if len(result) < number:
                    result.append(person)
                
        return result
               
    def assignPositionsRandom(self):
        for i in range(10000):
            d = self._chooseRandom(3, self.tm, "Defense", [])
            m = self._chooseRandom(3, self.tm, "Midfield", d)
            o = self._chooseRandom(3, self.tm, "Offense", m+d)
            
            os = [self._getPlayerName(x) for x in o] 
            offense_string = ",".join(os)

            ds = [self._getPlayerName(x) for x in d] 
            defense_string = ",".join(ds)
        
            ms = [self._getPlayerName(x) for x in m] 
            midfield_string = ",".join(ms) 
            
            # print( f"Team: {self.n}, Assigning players at random; attempt {i}")
            # print( f"  d: {len(d)} ({defense_string}), m: {len(m)} ({midfield_string}), o: {len(o)} ({offense_string})")
            
            if (len(o) == 2 or len(o) == 3) and len(d) == 3 and len(m) == 3:
                self.o = o
                self.d = d
                self.m = m
                return True
            
        print( "Failed")   
        return False
    
    def _printNamesInList(self, l):
        ls = [self._getPlayerName(x) for x in l]
        return ", ".join(ls)
    
    def _meetsCriteria(self, o, d, m):
        return len(o) + len(d) + len(m) == len(self.tm)
        
    def assignPositionsRec(self, players, o, d, m, max_to_assign=3, verbose=False):
        # Base case first.  All players have been selected.
        if len(players) == 0:
            return (o,d,m)
        
        p = players[0]
        if (len(players) > 1):
            remaining = players[1:]
        else:
            remaining = []
        
        availablePositions = p["Position"].split(",")
        if availablePositions[0] == "No preference":
            availablePositions = ["Offense", "Defense", "Midfield"]
          
        if verbose:
            print( f"assignPositionsRec; p: {p['Name']}, positions: {', '.join(availablePositions)}, o: {self._printNamesInList(o)}, d: {self._printNamesInList(d)}, m: {self._printNamesInList(m)}")
           
        maximum_assigned = sum(len(x) == max_to_assign for x in [o,d,m])
        
        for a in availablePositions:
            a = a.strip()
            if verbose:
                print( f"assignPositionsRec; now considering p: {p['Name']}, for a: {a}")
            if (a == "Offense" and (len(o)< max_to_assign or maximum_assigned >= 2) ):
                (ro, rd, rm) = self.assignPositionsRec(remaining, o + [p], d, m, max_to_assign, verbose)
                if self._meetsCriteria(ro, rd, rm):
                    return (ro,rd,rm)
            if (a == "Defense" and (len(d)< max_to_assign or maximum_assigned >= 2)):
                (ro, rd, rm) = self.assignPositionsRec(remaining, o, d + [p], m, max_to_assign, verbose)
                if self._meetsCriteria(ro, rd, rm):
                    return (ro,rd,rm)
            if (a == "Midfield" and (len(m)< max_to_assign or maximum_assigned >= 2) ):
                (ro, rd, rm) = self.assignPositionsRec(remaining, o, d, m + [p], max_to_assign, verbose)
                if self._meetsCriteria(ro, rd, rm):
                    return (ro,rd,rm)
                
            if verbose: 
                print( f"assignPositionsRec; it didn't work out for p: {p['Name']}, for a: {a}")
        if verbose:
            print( f"assignPositionsRec; didn't work out for p: {p['Name']}, in any positions, o: {len(o)}, d: {len(d)}, m: {len(m)}")
        
        # This one won't work out but will return it anyway.
        return (o,d,m)
            
    def assignPositions(self, max_to_assign = 3, verbose = False):
        if verbose:
            print (f"assignPositions;  going to call initial; team: {self.n}, names: {self._printNamesInList(self.tm)}")
    
        # Recursive approach with initial list of offense, defense, and midfield 
        # as empty and starting with all players to consider. 
        (o, d, m) = self.assignPositionsRec(self.tm, [], [], [], max_to_assign, verbose)
        
        if self._meetsCriteria(o,d,m):
            self.o = o
            self.d = d
            self.m = m
        
            return True
        else:
            return False
    
    
    def __str__(self):     
        player_string = ""
        for t in self.tm:
            name = self._getPlayerName(t)
            player_string += f"  Player: {name}\n";

        os = [self._getPlayerName(x) for x in self.o] 
        offense_string = "  Offense: " + ",".join(os)

        ds = [self._getPlayerName(x) for x in self.d] 
        defense_string = "  Defense: " + ",".join(ds)
        
        ms = [self._getPlayerName(x) for x in self.m] 
        midfield_string = "  Midfield: " + ",".join(ms) 
        
        return f"Team #{self.n}\n  Goal: {self._getPlayerName(self.g)}\n{player_string}{offense_string}\n{defense_string}\n{midfield_string}"; 

In [150]:
goal_keepers = all_players[ all_players["Position"] == "Goal keeper" ]
print( f"Found goal keepers; count: {len(goal_keepers)}")

Found goal keepers; count: 10


In [151]:
# Shuffle the players but in a stable way

def doOne(random_state):
    players = shuffle(all_players, random_state=random_state)

    players.drop(players[players['Position'] == "Goal keeper"].index, inplace = True)
    # Get rid of Mindy since she has already been placed.
    players.drop(players[players['Name'] == "Mindy Au"].index, inplace = True)

    # print(f"Remaining players to be placed: {len(players)}")
    # print(f"all_players: {len(all_players)}")

    teams = []
    count = 0
    for index, g in goal_keepers.iterrows():
        count += 1
        t = Team(count, g)

        if g["Name"] == "Maha":
            t.addPlayer(mindy)

        teams.append(t)

    # The goal here is to take the SNV - or "Skill Numeric Value" - and 
    # evenly distribute across all of the teams the different skill levels.
    # But we don't want to do this an overload any one team with 1s or 3s.
    snv_next_team_number = { 1 : 1, 2 : 1, 3 : 1}
    number_teams = len(goal_keepers)
    max_players_in_team = ceil(len(players) / float(number_teams))
    team_count = dict(zip(range(1,number_teams+1), [1]*number_teams)) 

    def getNextTeam(snv):
        f = snv_next_team_number[snv]
        while team_count[f] > max_players_in_team:
            f = f + 1
            if f == number_teams + 1:
                f = 1
        
        result = f
        team_count[f] += 1
        
        f = f + 1
        if f == number_teams + 1:
            f = 1
        snv_next_team_number[snv] = f
        return result

    # Create an array where we assign each skill level to the next available
    # team.  This ensures proper distribution of skills across all of the teams.
    team = [ getNextTeam(players.loc[x,"SNV"]) for x in players.index ]

    # Display some information about the distribution of players to teams.
    # h,_ = np.histogram(team, bins=len(goal_keepers))
    # print( f"Histogram is: {h})")
    
    # Add a team column to the players
    players.insert(1, "Team", value = team)

    for index, p in players.iterrows():
        teams[p["Team"]-1].addPlayer(p)

    number_works = 0
    for t in teams:
        result = t.assignPositions()
        if result:
            number_works += 1

    return (number_works, teams)

In [152]:
n, r = doOne(0)


In [153]:

m = 0
best_i = 0
best_teams = None
for i in range(100):
    (n, teams) = doOne(i)
    if n > m:
        best_i = i
        m = n
        best_teams = teams
    
    if i % 10 == 0:
        print (f"Done with {i}, best is {best_i} - with {m} good teams.")
    

print (f"Best teams is {best_i} and has {m} good teams")
for t in best_teams:
    print(str(t))
    

Done with 0, best is 0 - with 10 good teams.
Done with 10, best is 0 - with 10 good teams.
Done with 20, best is 0 - with 10 good teams.
Done with 30, best is 0 - with 10 good teams.
Done with 40, best is 0 - with 10 good teams.
Done with 50, best is 0 - with 10 good teams.
Done with 60, best is 0 - with 10 good teams.
Done with 70, best is 0 - with 10 good teams.
Done with 80, best is 0 - with 10 good teams.
Done with 90, best is 0 - with 10 good teams.
Best teams is 0 and has 10 good teams
Team #1
  Goal: Suzie Ness
  Player: Eva Gillespie
  Player: Fio Bazo
  Player: Blythe Simmons
  Player: Kristen Mittelsteadt
  Player: Jen Blume
  Player: Kimberly Driscoll
  Player: Heather Brammer
  Player: Lorna Croswell
  Player: Jennifer Sonson
  Offense: Fio Bazo,Blythe Simmons,Jen Blume
  Defense: Eva Gillespie,Kristen Mittelsteadt,Kimberly Driscoll,Heather Brammer
  Midfield: Lorna Croswell,Jennifer Sonson
Team #2
  Goal: Rachel Emswiler
  Player: Nev Trakic
  Player: Lindsay griffin
  Pla

In [154]:

for t in best_teams:
    print(f"\nTeam {t.n}")
    print(f"{t._getPlayerName(t.g)},Goal")
    if t.o:
        for o in t.o:
            print(f"{t._getPlayerName(o)},Offense")
        for o in t.m:
            print(f"{t._getPlayerName(o)},Midfield")
        for o in t.d:
            print(f"{t._getPlayerName(o)},Defense")
    else:
        for o in t.tm:
            print(f"{t._getPlayerName(o)},Unassigned,{o['Position']}")


Team 1
Suzie Ness,Goal
Fio Bazo,Offense
Blythe Simmons,Offense
Jen Blume,Offense
Lorna Croswell,Midfield
Jennifer Sonson,Midfield
Eva Gillespie,Defense
Kristen Mittelsteadt,Defense
Kimberly Driscoll,Defense
Heather Brammer,Defense

Team 2
Rachel Emswiler,Goal
Nev Trakic,Offense
Ashley Kangas,Offense
Mikala Zinter,Offense
Taren Beck,Offense
Alicia Castenada,Midfield
Lily Cornell,Midfield
Lindsay griffin,Defense
Katherine Chertok,Defense
Sneha Sastry,Defense

Team 3
Diana Roll,Goal
Carolyn Hall,Offense
Emily Ford,Offense
Danii bowlden,Midfield
Heather Gallemore “Galle”,Midfield
Beth Bronger-Jones,Midfield
Stephanie Howard,Defense
Max McCullar,Defense
Anna Claire Ferrer,Defense
Marissa London,Defense

Team 4
Christina Toney,Goal
Jessica Bierhaus,Offense
Deirdre Noonan,Offense
RENATA PEITL,Offense
Margaret Fraczek,Midfield
Amanda Smith,Midfield
Lindsey Hewes,Defense
Barb Marquardt,Defense
Genevieve Norwood,Defense
Diana Shpektor,Defense

Team 5
Heather Johnson,Goal
Mason Alfona,Offense
Ph

In [88]:
best_teams[8].assignPositions()

True